In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## 1. plan of attack.
 **1. data cleaning**<br>
 **2. EDA(Exploretory Data Analysis)**<br>
 **3. text preprocessing**<br>
 **4. Model Building**<br>
 **5. Model Evaluation**<br> 
 **6. Improvement**<br>

# 1. Introduction and data cleaning 

In [ ]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding='ISO-8859-1')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#drop last 3 columns
df.drop(columns = ['Unnamed: 2','Unnamed: 3','Unnamed: 4'], inplace = True)
df.head()

In [ ]:
# rename the columns 
df.rename(columns= {'v1':'target','v2':'text'},inplace = True)
df.head()

In [ ]:
df.head()

In [ ]:
#check for duplicate
df.duplicated().sum()

In [ ]:
df.drop_duplicates(keep = 'first',inplace = True)

In [ ]:
# apply label incoder ham = 0, spam = 1
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['target']=encoder.fit_transform(df['target'])

In [ ]:
df.head()

# 2. DATA ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
plt.pie(df['target'].value_counts(),labels = ['ham','spam'],autopct = '%0.2f')
plt.title('Ham vs Spam')
plt.show()

* "The dataset exhibits class imbalance, and I will implement appropriate strategies to address this imbalance during the model buliding phase."


In [ ]:
df['length'] = df['text'].apply(len)

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.title('ham')
sns.histplot(df[df['target']==0]['length'])
plt.subplot(1,2,2)
plt.title('spam')
sns.histplot(df[df['target']==1]['length'])

* "It's noteworthy that spam messages tend to be longer in length compared to ham messages. Specifically, the majority of ham messages fall within the 1-100 character range, while a significant portion of spam messages exceed 100 characters in length."

# 3. Data preprocessing
1. lower case
2. Tokenization (convert into list).
3. Removing special charecters(like- %,&,@).
4. Removing stopwords and punchuation( like - am,are,is,you,them)
5. Stemming( loving,loved,love = 'love')

In [ ]:
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    text = y.copy()
    y.clear()
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    text = y.copy()
    y.clear()
    for i in text:
        y.append(ps.stem(i))
    return " ".join(y)
    

In [ ]:
df['transformed_text'] = df['text'].apply(transform_text)

In [ ]:
df.head()

#### I. most frequent words used in spam masseges.

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(width = 500, height = 500, min_font_size = 10, background_color = 'white')

In [ ]:
spam_wc = wc.generate(df[df['target']==1]['transformed_text'].str.cat(sep = " "))
plt.title("Most frequently words used in spam masseges.")
plt.imshow(spam_wc)


#### II. Most frequent words used in Normal masseges.

In [ ]:
spam_wc = wc.generate(df[df['target']==0]['transformed_text'].str.cat(sep = " "))
plt.title("Most frequently words used in normal masseges.")
plt.imshow(spam_wc)

# 4. Model building

### 1. first of all I will try Naive Bayes as it gives good result on textual data. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features = 2500)

In [ ]:
x = tfidf.fit_transform(df['transformed_text']).toarray()
y = df['target'].values

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [ ]:
print('gaussian Naive Bayes')
gnb.fit(x_train,y_train)
y_pred1 = gnb.predict(x_test)
print(accuracy_score(y_test,y_pred1))
print(precision_score(y_test,y_pred1))
print("*"*25)
print('Multinomial Naive Bayes') 
mnb.fit(x_train,y_train)
y_pred2 = mnb.predict(x_test)
print(accuracy_score(y_test,y_pred2))
print(precision_score(y_test,y_pred2))
print("*"*25)
print('bernoulli naive Bayes')
bnb.fit(x_train,y_train)
y_pred3 = bnb.predict(x_test)
print(accuracy_score(y_test,y_pred3))
print(precision_score(y_test,y_pred3))

### 2. Multinomial Naive Bayes is giving good result but we should try others as well.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier


In [ ]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)

In [ ]:
clfs = {
    'SVC' : svc,
    'KN' : knc, 
    'NB': mnb, 
    'DT': dtc, 
    'LR': lrc, 
    'RF': rfc, 
    'AdaBoost': abc, 
    'BgC': bc, 
    'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
}

In [ ]:
def train_classifier(clf,x_train,y_train,x_test,y_test):
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    
    return accuracy,precision

In [ ]:
accuracy_scores = []
precision_scores = []

for name,clf in clfs.items():
    
    current_accuracy,current_precision = train_classifier(clf, x_train,y_train,x_test,y_test)
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)
    
performance_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy':accuracy_scores,'Precision':precision_scores}).sort_values('Precision',ascending=False)
performance_df

 3. We have observed that the K-Nearest Neighbors (KNeighbors), Multinomial Naive Bayes (MultinomialNB), and rfc(random forest) algorithms consistently yield high scores and demonstrate superior precision in our machine learning experiments. To harness the strengths of these individual classifiers, we intend to explore ensemble techniques, specifically the use of a `voting algorithm`, to further enhance predictive performance.<br><br>
**what is voting algorithm**
* In machine learning, a "voting algorithm" refers to an ensemble learning technique where multiple individual machine learning models are combined to make predictions or decisions. The idea behind a voting algorithm is to leverage the diversity of these individual models to improve the overall predictive accuracy and reliability

In [ ]:
#voting classifier
knc = KNeighborsClassifier()
mnb = MultinomialNB()
rfc = RandomForestClassifier(n_estimators=50, random_state=2)

In [ ]:
from sklearn.ensemble import VotingClassifier
voting = VotingClassifier(estimators=[('knc',knc),('nb',mnb),('rfc',rfc)])

In [ ]:
voting.fit(x_train,y_train)

In [ ]:
y_pred = voting.predict(x_test)
print("Accuracy",accuracy_score(y_test,y_pred))
print("Precision",precision_score(y_test,y_pred))

### 4. voting algorithm didn't performed well so finally we choose `Multinomial Naive Bayes`.

In [ ]:
fitted_mnb = mnb.fit(x_train,y_train)

In [ ]:
import pickle 
pickle.dump(tfidf,open('vectorizer.pkl','wb'))
pickle.dump(fitted_mnb,open('mnb_model.pkl','wb'))